In [34]:
import pandas as pd
import os

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

from tkinter import messagebox

import re
import time

In [35]:
print('正在打开……')
df_mr=pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

datapath1 = filedialog.askopenfilename()

正在打开……


### 读取源数据

In [36]:
df1=pd.read_excel(datapath1,sheet_name='终端购机',dtype='str')
df2=pd.read_excel(datapath1,sheet_name='无宽客户',dtype='str')
df3=pd.read_excel(datapath1,sheet_name='全家享2.0',dtype='str')

将中文是和否，替换为0和1

In [37]:
df1['是否为宽带客户'] = df1['是否为宽带客户'].map({'无宽客户':0, '有宽客户':1})

### 备份

In [38]:
df1_bck=df1.copy()
df2_bck=df2.copy()
df3_bck=df3.copy()

### 还原

In [94]:
df1=df1_bck
df2=df2_bck
df3=df3_bck

### 读取派单规则

In [39]:
print('正在打开……')
df_mr=pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

datapath_r = filedialog.askopenfilename()

正在打开……


In [40]:
df_r1=pd.read_excel(datapath_r,sheet_name='渠道')
df_r2=pd.read_excel(datapath_r,sheet_name='直销')
df_r3=pd.read_excel(datapath_r,sheet_name='三大家')

### 数据清理

将不是数字类型的数字转化为数字类型。

In [41]:
# df1[['服务中心ID','网格ID']] = df1[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# df2[['服务中心ID','网格ID']] = df2[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# df3[['服务中心ID','网格ID']] = df3[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# 'raise'参数：无效的解析将引发异常
# 'corece'参数：将无效解析设置为NaN
# 'ignore'参数：无效的解析将返回输入,也就是遇到无效数据不解析，也不会填充NaN或者0
# 用ignore保留中文
df1 = df1.apply(pd.to_numeric, errors='ignore').fillna(0.0)
df2 = df2.apply(pd.to_numeric, errors='ignore').fillna(0.0)
df3 = df3.apply(pd.to_numeric, errors='ignore').fillna(0.0)

### 递归，根据各种不同条件，取满足条件的号码，并依次遍历

实际上就是将get和set改变为装饰器，也就是点访问

df__t_tmp 满足条件的号码池
pdl 派单量
row 打标，分配编码
stack 堆栈，存储满足各种条件的号码池
df_res_total 存放本次循环的结果
df_res 取满足派单量的数据
df_orgin 原始df

In [42]:
#取出各条件号码，依次遍历

def handle(df_t_tmp, pdl, row, queue, df_res_total, df_res,df_orgin):
    # print(len(df_t_tmp))
    # 结束条件df_end长度
    if (len(df_t_tmp) == 10000):
        return df_res_total

    if ((len(df_t_tmp) >= pdl)):
        df_res = df_t_tmp.head(pdl).copy()
        df_orgin=df_orgin[~df_orgin.号码.isin(df_res.号码)]
        df_res['打标'] = row
        df_res_total = pd.concat([df_res_total, df_res])
        # print(df_res_total)
        return df_res_total
    else:
        last_pdl = pdl - len(df_t_tmp)
        df_res = df_t_tmp.head(len(df_t_tmp)).copy()
        df_orgin=df_orgin[~df_orgin.号码.isin(df_res.号码)]
        df_res['打标'] = row
        df_res_total = pd.concat([df_res_total, df_res])
        # print(df_res_total)
        df_t_tmp = queue.pop(0)  # 队列出队
        return handle(df_t_tmp, last_pdl, row, queue, df_res_total, df_res,df_orgin)

### 渠道

条件对应二进制码

无宽客户

In [43]:
term=[15,14,13,11,12,10,9,8,7,6,5,3,4,2,1,0]

全家享

In [44]:
qjx_term=[7,6,5,4,3,2,1,0]

终端购机

In [45]:
zdgj_term=[0,1]

条件结构体

In [46]:
class wkkh_term(object):

    def __init__(self, a, b,c,d):

        self.a = int(a)

        self.b = int(b)

        self.c = int(c)
        
        self.d = int(d)


    def df_wk(self,df_tmp):
        df_tmp_res=df_tmp[(df_tmp['80以上资费高价值客户']==self.a)&(df_tmp['我号异宽']==self.b)&(df_tmp['终端换机']==self.c)&(df_tmp['4G资费']==self.d)]
        return df_tmp_res
    
    def df_qjx(self,df_tmp):
        df_tmp_res=df_tmp[(df_tmp['是否裸奔高价值']==self.a)&(df_tmp['是否质差客户']==self.b)&(df_tmp['有免费成员空间']==self.c)]
        return df_tmp_res
    def df_zdgj(self,df_tmp):
        df_tmp_res=df_tmp[(df_tmp['是否为宽带客户']==self.a)]
        return df_tmp_res

清理重复号码函数

In [47]:
def df_clean(df,df_no_need):
        df=df[~df.号码.isin(df_no_need.号码)]
        return df


### 条件进队

In [49]:
def choose_consumer(type,ob,df_tmp):
    if '无宽' in type:
        return wkkh_term.df_wk(ob,df_tmp)

    if '全家享' in type:
        return    wkkh_term.df_qjx(ob,df_tmp) 

    if '终端购机' in type:
        return    wkkh_term.df_zdgj(ob,df_tmp) 

In [50]:
def choose_term(type):
    if '无宽' in type:
        return term

    if '全家享' in type:
        return   qjx_term

    if '终端购机' in type:
        return   zdgj_term

In [51]:
def get_kh(df_tmp,consumer_type,type,term):
    #consumer_type为客户类型
    df_stack = []
    for i in term:
        i=i<<consumer_type#移动几位
        res='{:04b}'.format(i)
        a=res[0]
        b=res[1]
        c=res[2]
        d=res[3]
        ob=wkkh_term(a,b,c,d)
        df_res=choose_consumer(type,ob,df_tmp)
            
        df_stack.append(df_res)
    #结束条件
    df_end = pd.Series(range(0, 10000))
    df_stack.append(df_end)
    return df_stack

分配函数

In [59]:
def divide(df_rule,out_path,r_id,type,df_origin):
        global df1
        global df2
        global df3
        df_res_total=pd.DataFrame()
        df_res_total_l=pd.DataFrame()

        df_stack = []

        consumer_dict=dict({'无宽': 0, '全家享': 1, '终端': 2})
        consumer_type=0
        for key in consumer_dict:
                if type in key:
                    consumer_type=consumer_dict[key]
                    break
        #consumer_type移位所需位数,type判断获得哪类类型客户号码
        term_now=choose_term(type)

        for index, row in df_rule.iterrows():
            #取出各条件号码，依次遍历
            df_res=pd.DataFrame()
            df_tmp=df_origin[df_origin[r_id]==row[r_id]]
            pdl1 = row[type]#修改条件
            df_stack = []

            
            df_stack = get_kh(df_tmp,consumer_type,type,term_now)

            # print(len(df_end))
            #从2开始，有n个条件，就压栈n-1个条件和df_end条件
            pdl1 = int(pdl1)
            row = row['派单编号']
            #压出队头，利用队列思想
            df_t_tmp=df_stack.pop(0)
            # 递归
            df_res = handle(df_t_tmp, pdl1, row, df_stack, df_res_total, df_res,df_origin)
            df_res_total_l = pd.concat([df_res_total_l, df_res])
            df_origin=df_clean(df_origin,df_res_total_l)

        df_res_total_l.to_excel(out_path)
        df3=df3[~df3.号码.isin(df_res_total_l.号码)]
        df2=df2[~df2.号码.isin(df_res_total_l.号码)]
        df1=df1[~df1.号码.isin(df_res_total_l.号码)]

### 开始分配

### 渠道

无宽客户

In [60]:
outpath='渠道-无宽客户-结果.xlsx'
divide(df_r1,outpath,'网格ID','无宽客户',df2)


0


全家享2.0

In [61]:
outpath='渠道-全家享2.0-结果.xlsx'
divide(df_r1,outpath,'服务中心ID','全家享2.0',df3)


0


### 直销

无宽客户

In [71]:
outpath='直销-无宽客户-结果.xlsx'
divide(df_r2,outpath,'服务中心ID','无宽客户',df2)


全家享

In [72]:
outpath='直销-全家享2.0-结果.xlsx'
divide(df_r2,outpath,'服务中心ID','全家享2.0',df3)


### 三大家


终端购机

In [74]:
outpath='三大家-终端购机-结果.xlsx'
divide(df_r3,outpath,'服务中心ID','终端购机',df1)


全家享

In [ ]:
outpath='三大家-全家享-结果.xlsx'
divide(df_r3,outpath,'服务中心ID','全家享2.0',df3)
